# Analisi esplorativa e clustering

Questo notebook presenta l'analisi esplorativa del training set.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import matplotlib.cm as cm
import seaborn as sns

import tqdm

from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import sklearn.metrics.pairwise as pw

# %matplotlib inline

export_images = True
style = 'white'
img_folder = './img/'

# tune this for bigger figures
pl.rcParams['figure.figsize'] = (14, 14)
sns.set_context('notebook')

Carichiamo il dataset:

In [ ]:
dataset_raw = pd.read_excel("./Dataset/finali/integrato_2014.xlsx", sheet_name='ML_finale')
dataset_raw.head()

Per comodità rinominiamo le colonne:

In [ ]:
dataset_raw = dataset_raw.rename(columns={'CO2 production (kg)': 'CO2',
                           'Charcoal consumption (kg)': 'Charcoal',
                           'Fuel oil consumption (kg)': 'Fuel oil',
                           'Renewable energy consumption (percentage)': 'Clean energy',
                           'PM2.5 (micrograms)': 'PM2.5'})
dataset_raw.head()

In [ ]:
# Plot the original dataset
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')

    fig, ax = pl.subplots(3, 2, figsize=(18, 14))
    plot_kde = False
    sns.distplot(dataset_raw['CO2'], ax=ax[0, 0], kde=plot_kde)
    sns.distplot(dataset_raw['Charcoal'], ax=ax[0, 1], kde=plot_kde)
    sns.distplot(dataset_raw['Fuel oil'], ax=ax[1, 0], kde=plot_kde)
    sns.distplot(dataset_raw['Clean energy'], ax=ax[1, 1], kde=plot_kde)
    sns.distplot(dataset_raw['PM2.5'], ax=ax[2, 0], kde=plot_kde)
    sns.distplot(dataset_raw['GDP'], ax=ax[2, 1], kde=plot_kde)
    sns.despine()
    
    fig2, ax2 = pl.subplots(figsize=(18, 10))
    sns.distplot(dataset_raw['Population'], ax=ax2, kde=plot_kde)
    sns.despine()
    
    pl.show()

    if export_images:
        fig.savefig(img_folder + 'dataset-originale.png', bbox_inches='tight')
        fig2.savefig(img_folder + 'dataset-originale-popolazione.png', bbox_inches='tight')
        sns.set_context('notebook')

Vediamo le distribuzioni appaiate dei dati originali:

In [ ]:
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')
    pairplot = sns.pairplot(dataset_raw)
    pl.show()
    
    if export_images:
        fig = pairplot.fig
        fig.savefig(img_folder + 'pairplot-originale.png', bbox_inches='tight')
        sns.set_context('notebook')

Normalizziamo gli attributi rispetto alla popolazione:

In [ ]:
def normalize_with_col(dataset, ref_column=None, cols_to_normalize=None):
    """Divide all columns in a dataset for the normalization column."""
    if ref_column is None:
        raise ValueError("Must choose a reference column to normalize.")
    if cols_to_normalize is None:
        raise ValueError("Must select target columns.")

    norm_col = dataset[ref_column]
    result = dataset.copy()
    
    for col in cols_to_normalize:
        result[col] = result[col] / norm_col
    
    return result

In [ ]:
dataset = normalize_with_col(dataset_raw,
                             ref_column='Population',
                             cols_to_normalize=['CO2', 'Charcoal', 'Fuel oil', 'GDP'])
# Peek at the data
dataset.head()

## Statistiche descrittive per il dataset

Il dataset contiene 153 istanze di paesi, ognuna con gli attributi

- popolazione
- produzione di CO_2 annuale (in kg)
- consumo di carbone annuale (in kg)
- consumo di carburanti fossili annuale (in kg)
- percentuale di energia rinnovabile utilizzata, rispetto all'utilizzo totale di quel paese
- GDP (prodotto interno lordo)

Vediamo come sono distribuite le variabili.

In [ ]:
dataset.describe()

Ora vediamo la distribuzione delle features:

In [ ]:
# Plot the population-scaled dataset
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')

    fig, ax = pl.subplots(3, 2, figsize=(18, 14))
    plot_kde = False
    sns.distplot(dataset['CO2'], ax=ax[0, 0], kde=plot_kde)
    sns.distplot(dataset['Charcoal'], ax=ax[0, 1], kde=plot_kde)
    sns.distplot(dataset['Fuel oil'], ax=ax[1, 0], kde=plot_kde)
    sns.distplot(dataset['Clean energy'], ax=ax[1, 1], kde=plot_kde)
    sns.distplot(dataset['PM2.5'], ax=ax[2, 0], kde=plot_kde)  # TODO: sistemare labels
    sns.distplot(dataset['GDP'], ax=ax[2, 1], kde=plot_kde)
    sns.despine()
    
    fig2, ax2 = pl.subplots(figsize=(18, 10))
    sns.distplot(dataset['Population'], ax=ax2, kde=plot_kde)
    sns.despine()
    
    pl.show()

    if export_images:
        fig.savefig(img_folder + 'dataset-scalato-popolazione.png', bbox_inches='tight')
        sns.set_context('notebook')

In [ ]:
def standardize(dataset, cols_to_standardize=None):
    """Standardize dataset.
    
    Returns a new copy of the dataset with the
    selcted columns standardized.
    """
    if cols_to_standardize is None:
        raise ValueError("No column passed for standardization")

    result = dataset.copy()
    for col in cols_to_standardize:
        vals = preprocessing.scale(dataset[col].values)
        result.loc[:, col] = vals

    return result

In [ ]:
dataset_std = standardize(dataset,
                          cols_to_standardize=[
                              'Population',
                              'CO2',
                              'Charcoal',
                              'Fuel oil',
                              'Clean energy',
                              'PM2.5',
                              'GDP'])
dataset_std.head()

In [ ]:
# Plot the standardized population-scaled dataset
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')

    fig, ax = pl.subplots(3, 2, figsize=(18, 14))
    plot_kde = False
    sns.distplot(dataset_std['CO2'], ax=ax[0, 0], kde=plot_kde)
    sns.distplot(dataset_std['Charcoal'], ax=ax[0, 1], kde=plot_kde)
    sns.distplot(dataset_std['Fuel oil'], ax=ax[1, 0], kde=plot_kde)
    sns.distplot(dataset_std['Clean energy'], ax=ax[1, 1], kde=plot_kde)
    sns.distplot(dataset_std['PM2.5'], ax=ax[2, 0], kde=plot_kde)  # TODO: sistemare labels
    sns.distplot(dataset_std['GDP'], ax=ax[2, 1], kde=plot_kde)
    sns.despine()
    pl.show()

    if export_images:
        fig.savefig(img_folder + 'dataset-std.png', bbox_inches='tight')
        sns.set_context('notebook')

In [ ]:
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')
    pairplot = sns.pairplot(dataset_std.loc[:,
                                           ['CO2',
                                           'Charcoal',
                                           'Fuel oil',
                                           'Clean energy',
                                           'PM2.5',
                                           'GDP']])
    pl.show()
    
    if export_images:
        fig = pairplot.fig
        fig.savefig(img_folder + 'pairplot-standardizzato.png', bbox_inches='tight')
        sns.set_context('notebook')

## Correlazione tra features

Esploriamo la correlazione tra le features con una heatmap:

In [ ]:
def create_heatmap(dataset, figsize=(14, 14)):
    """Create a heatmap from the dataset."""
    # Compute the correlation matrix
    corr = dataset.corr()

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = pl.subplots(figsize=figsize)
    
    # Generate a custom diverging colormap
    cmap = sns.color_palette("RdBu")
    
    # Draw the heatmap with the mask and correct aspect ratio
    hm = sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, center=0, cmap=cmap, annot=True,
                     square=True, linewidths=.5, cbar_kws={"shrink": .5})
    return hm

In [ ]:
if export_images:
    sns.set_context('talk')
    heatmap = create_heatmap(dataset_raw)
    fig = heatmap.figure
    fig.savefig(img_folder + 'correlazione.png', bbox_inches='tight')
    sns.set_context('notebook')

# K-Means con Scikit-Learn

Ora che abbiamo esplotato il dataset, utilizziamo Sklearn per indurre un modello K-Means.

### Selezione delle feature

Da quanto emerge dall'analisi esplorativa, la feature "Fuel Oil Consumption" è estremamente sbilanciata verso lo zero, con solo l'Afghanistan a superare quota 10 kg per-capita.

In base a ciò, si è deciso di **eliminare la feature e ritenere soltanto le altre**, scalate rispetto alla popolazione e standardizzate.

In [ ]:
data_km = dataset_std.copy()
data_km = data_km.drop(columns=['Fuel oil', 'Population'])
data_km.head()

### Pair-wise plot delle feature scelte


In [ ]:
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')
    pairplot = sns.pairplot(data_km, size=2.5)
    pl.show()
    
    if export_images:
        fig = pairplot.fig
        fig.savefig(img_folder + 'pairplot-KMeans.png', bbox_inches='tight')
        sns.set_context('notebook')

## Clustering

Ora che il dataset è standardizzato, procediamo a clusterizzare.

Per trovare la migliore clusterizzazoine, creiamo una funzione.

In [ ]:
def clusterize(X, min_clusters, max_clusters):
    n_clusters = list(range(min_clusters, max_clusters + 1))
    
    # (i, j) -> label for data point i when using n_clusters[j] clusters
    labels = np.zeros((X.shape[0], len(n_clusters)), dtype=np.int16)
    
    # (0, j) -> average silhouette score when using n_clusters[j] clusters
    silh_tot = np.zeros((1, len(n_clusters)))
    
    # (i, j) -> silhouette score for data point i when using n_clusters[j] clusters
    silh_ith = np.zeros((X.shape[0], len(n_clusters)))
    
    for ind, n in enumerate(tqdm.tqdm_notebook(n_clusters)):
        clusterer = KMeans(n_clusters=n, init='k-means++', n_init=50,
                           max_iter=100000, tol=1e-7,
                           precompute_distances=True, random_state=10,
                           n_jobs=1)
        
        curr_labels = clusterer.fit_predict(X)
        labels[:, ind] = curr_labels

        silhouette_avg = silhouette_score(X, curr_labels)
        silh_tot[0, ind] = silhouette_avg

        curr_silhouette_values = silhouette_samples(X, curr_labels)
        silh_ith[:, ind] = curr_silhouette_values

    ret_labels = pd.DataFrame(data=labels, columns=n_clusters)
    ret_silh_avg = pd.DataFrame(data=silh_tot, columns=n_clusters)
    ret_silh_point = pd.DataFrame(data=silh_ith, columns=n_clusters)

    return ret_labels, ret_silh_avg, ret_silh_point

Ora lanciamo il K-Means su un numero variabile di clusters, da 2 a 100. Vediamo poi quale è il migliore con l'indice di silhouette

In [ ]:
data_values = data_km[['CO2', 'Charcoal', 'Clean energy', 'PM2.5', 'GDP']].values
labels, avg_silhouette, point_silhouette = clusterize(data_values, 2, 20)

In [ ]:
best_index = np.argmax(avg_silhouette.values)
best_silhouette = avg_silhouette.iloc[0, best_index]
best_n_cluster = avg_silhouette.columns[best_index]

best_cluster_labels = labels[labels.columns[best_index]].values

print("Best silhouette score is {:6.4f} with {} clusters"
     .format(best_silhouette, best_n_cluster))

In [ ]:
### Plot andamento silhouette media con il numero di clusters
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')

    avg_silh_plot = avg_silhouette.loc[0, :].values
    fig, ax = pl.subplots(ncols=1, figsize=(14, 10))
    pl.plot(np.array(avg_silhouette.columns.tolist()), avg_silh_plot)
    ax.set_title("Silhouette score")
    sns.despine()
    
    pl.show()
    
    if export_images:
        fig.savefig(img_folder + 'silhouette_vs_nclusters.png', bbox_inches='tight')
        sns.set_context('notebook')

### Plot dei valori di silhouette

Definiamo una funzione per plottare la silhouette:

In [ ]:
# define a function that outputs the silhouette values for each cluster
def silhouette_for_clusters(labels, silhouettes):
    if len(labels) != len(silhouettes):
        raise ValueError("Lenght of labels ({}) differs from length of silhouettes ({})"
                        .format(len(labels), len(silhouettes)))

    cluster_names, cluster_size = np.unique(labels, return_counts=True)
    cluster_silh = {n: np.mean(silhouettes[labels == n]) for n in cluster_names}
    
    return cluster_silh, cluster_size

In [ ]:
cluster_silh, cluster_size = silhouette_for_clusters(
    best_cluster_labels,
    point_silhouette[best_n_cluster].values
)

In [ ]:
print("Ci sono {} clusters.\n".format(len(cluster_size)))
for ind, size in enumerate(cluster_size):
        print("Cluster {} ha {:3d} elementi e silhouette = {:6.4f}".format(ind, size, cluster_silh[ind]))

In [ ]:
def plot_silhouette(cluster_labels, avg_silh, point_silh, cluster_silh, n_clusters, fig, ax, context):
    """Make a silhouette plot.
    
    Parameters
    ----------
    data: pd.DataFrame
        data corresponding to the plots
        
    cluster_labels: np.array of shape (n_points, 1)
        labels of the cluster for each data point
    
    point_silh: np.array (n_points, 1)
        silhouette for each data point at the defined cluster number n_clusters
    
    avg_silh: float
        average silhouette score for this clusterization
    """
    min_silh_score = point_silh.min()
    max_silh_score = point_silh.max()
    
    ax.set_xlim([min_silh_score, max_silh_score])
    ax.set_ylim([0, len(cluster_labels) + (n_clusters + 1) * 10])
    
    y_lower = 10
    
    for i in range(n_clusters):
        # aggregate silhouette by cluster label and sort it
        silh_values_cluster_i = point_silh[cluster_labels == i]
        silh_values_cluster_i.sort()
        
        ith_cluster_size = silh_values_cluster_i.shape[0]
        y_upper = y_lower + ith_cluster_size
        
        color = cm.spectral(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                        0, silh_values_cluster_i,
                        facecolor=color, edgecolor=color, alpha=0.7)
        
        # Label with the cluster number in the middle
        text_x = -0.09 if context != 'talk' else -0.11
        text = "Cluster {}\nsilhouette = {:4.2f}".format(str(i), cluster_silh[i])
        ax.text(text_x, y_lower + 0.5 * ith_cluster_size, text)
        y_lower = y_upper + 10
    
    ax.set_title("Silhouette plot con {} clusters".format(n_clusters))
#    ax.set_yticks([])
    ax.axvline(x=avg_silh, color='blue', linestyle='--')
    
    return fig, ax

In [ ]:
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')

    fig, ax = pl.subplots(ncols=1, figsize=(16, 20))
    plot_silhouette(labels[best_n_cluster].values,
                    avg_silhouette[best_n_cluster].values,
                    point_silhouette[best_n_cluster].values,
                    cluster_silh,
                    best_n_cluster,
                    fig, ax, 'talk')
    sns.despine()
    ax.spines['left'].set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    pl.show()
    
    if export_images:
        fig.savefig(img_folder + 'Silhouette.png', bbox_inches='tight')
        sns.set_context('notebook')

Aggiungo la colonna con i labels al dataset:

In [ ]:
data_final = data_km.copy()
data_final['cluster'] = best_cluster_labels

In [ ]:
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')
    pairplot = sns.pairplot(data_final,
                            vars=['CO2', 'Charcoal', 'Clean energy', 'PM2.5', 'GDP'],
                            hue='cluster')
    pl.show()
    
    if export_images:
        fig = pairplot.fig
        fig.savefig(img_folder + 'pairplot-KMeans-con-clusters.png', bbox_inches='tight')
        sns.set_context('notebook')

In [ ]:
def create_similarity_matrix(data, output='distance'):
    """Create a dissimilarity matrix from the input data,
    considered as (n_samples, n_features)
    """
    distances = pw.pairwise_distances(data, metric='euclidean', n_jobs=1)
    similarity = np.max(distances) - distances
    return similarity
    
def create_distance_matrix(data):
    """Create a distance matrix from the data, considered as (n_samples, n_features)."""
    distances = pw.pairwise_distances(data, metric='euclidean', n_jobs=1)
    return distances

Creiamo la matrice di dissimilarità (o delle distanze):

In [ ]:
dissimilarity_matrix = create_distance_matrix(data_values)
similarity_matrix = create_similarity_matrix(data_values)

Plottiamo la matrice.

In [ ]:
# data_values is the variable containing the data to measure dissimilarity
fig, ax = pl.subplots(figsize=(16, 16))
sns.heatmap(dissimilarity_matrix, square=True,
            cmap=sns.cubehelix_palette(8, start=0.5, rot=-0.75), ax=ax)
pl.show()

In [ ]:
# data_values is the variable containing the data to measure similarity
fig, ax = pl.subplots(figsize=(16, 16))
sns.heatmap(similarity_matrix, square=True,
            cmap=sns.cubehelix_palette(8, start=0.5, rot=-0.75), ax=ax)
pl.show()

## Analisi PCA per la varianza spiegata

Vediamo il contributo di ogni feature alla varianza spiegata dei dati:

In [ ]:
pca = PCA(n_components=None)
X_pca = pca.fit_transform(dataset_std.loc[:,['CO2',
                                             'Charcoal',
                                             'Clean energy',
                                             'PM2.5',
                                             'GDP']])
pca.explained_variance_ratio_

In [ ]:
def plot_explained_variance(pca_obj, threshold=0.9, despine=False):
    n = len(pca_obj.explained_variance_ratio_)
    x_vals = np.arange(n)
    
    # bar chart
    pl.bar(x_vals,
           pca_obj.explained_variance_ratio_,
           alpha=0.5,
           align="center",
           label="Varianza spiegata attributo")
    
    # step plot
    pl.step(x_vals,
            np.cumsum(pca_obj.explained_variance_ratio_),
            where="mid",
            label="Varianza spiegata cumulata")
    
    # threshold
    p = pl.plot(x_vals,
           threshold * np.ones(x_vals.shape),
           linestyle='--',
           label="Soglia al {}%".format(threshold * 100))
    
    pl.xlabel("Componenti principali")
    pl.ylabel("Rapporto varianza spiegata")
    pl.legend(loc="center right")
    if despine:
        sns.despine()
    pl.show()

In [ ]:
with sns.axes_style(style):
    if export_images:
        sns.set_context('talk')
        
    fig, ax = pl.subplots(ncols=1, figsize=(12, 10))
    plot_explained_variance(pca, threshold=0.95, despine=True)
    
    if export_images:
        fig.savefig(img_folder + 'varianza-spiegata.png', bbox_inches='tight')
        sns.set_context('notebook')